In [ ]:
#| default_exp microclip

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pathlib import Path
import subprocess
from fastai.vision.all import *

In [ ]:
#| export
def create_microclips(dataroot_path, offset=120, fps=1, n_workers=4):
    """
    Create microclips (small mp4 files) from frames.

    Args:
    - `dataroot_path` (`str` or `Path`): Root path of the data directory.
    - `offset` (`int`): Number of seconds of video until the anchor frame.
    - `fps` (`int`): Frames per second for the extracted microclip.
    - `n_workers` (`int`): Number of worker processes to use.

    Returns:
    - `None`
    """
    dataroot_path = Path(dataroot_path)
    frames_folder = dataroot_path / 'frames'
    microclips_dir = dataroot_path / 'microclips'
    microclips_dir.mkdir(parents=True, exist_ok=True)
    temp_dir = dataroot_path / 'temp'
    temp_dir.mkdir(parents=True, exist_ok=True)

    frames = get_image_files(frames_folder)
    data = []
    for frame_path in frames:
        filename = frame_path.name
        surgery_name, video_name, timestamp = parse_filename(filename)
        data.append({
            'frame_path': frame_path,
            'surgery_name': surgery_name,
            'video_name': video_name,
            'timestamp': timestamp
        })

    parallel(process_microclip, data, microclips_dir=microclips_dir, dataroot_path=dataroot_path,
             offset=offset, fps=fps, n_workers=n_workers, temp_dir=temp_dir)

In [ ]:
#| export
def process_microclip(item, microclips_dir, dataroot_path, offset, fps, temp_dir):
    """
    Process a single microclip.

    Args:
    - `item` (`dict`): Dictionary containing frame info.
    - `microclips_dir` (`Path`): Directory to save microclips.
    - `dataroot_path` (`Path`): Root path of the data directory.
    - `offset` (`int`): Number of seconds of video until the anchor frame.
    - `fps` (`int`): Frames per second for the extracted microclip.
    - `temp_dir` (`Path`): Directory for temporary files.

    Returns:
    - `None`
    """
    frame_path = item['frame_path']
    filename = frame_path.name
    surgery_name = item['surgery_name']
    video_name = item['video_name']
    timestamp = item['timestamp']

    microclip_name = frame_path.stem + '.mp4'
    microclip_path = microclips_dir / microclip_name

    video_path = dataroot_path / 'videos' / surgery_name / f'{video_name}.mp4'

    if not video_path.exists():
        print(f"Video file '{video_path}' does not exist.")
        return

    start_time = max(0, timestamp - offset)
    duration = timestamp - start_time

    if duration <= 0:
        print(f"Invalid duration for {video_path}: start_time={start_time}, timestamp={timestamp}")
        return

    temp_video_path = temp_dir / f'{frame_path.stem}_temp.mp4'

    extract_clip(video_path, start_time, duration, temp_video_path, fps)

    temp_video_path.replace(microclip_path)

In [ ]:
#| export
def extract_clip(video_path, start_time, duration, output_path, fps):
    """
    Extract a clip from the video.

    Args:
    - `video_path` (`Path`): Path to the source video.
    - `start_time` (`float`): Start time in seconds.
    - `duration` (`float`): Duration in seconds.
    - `output_path` (`Path`): Path to save the extracted clip.
    - `fps` (`int`): Frames per second for the extracted clip.

    Returns:
    - `None`
    """
    command = [
        'ffmpeg', '-ss', str(start_time), '-i', str(video_path),
        '-t', str(duration), '-vf', f'fps={fps}',
        '-c:v', 'libx264', '-an', '-y', str(output_path),
        '-hide_banner', '-loglevel', 'error'
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if result.returncode != 0:
        print(f"Failed to extract clip from {video_path} to {output_path}")

In [ ]:
#| export
def parse_filename(filename):
    """
    Parse the filename to extract surgery name, video name, and timestamp.

    Args:
    - `filename` (`str`): Filename to parse.

    Returns:
    - `surgery_name` (`str`)
    - `video_name` (`str`)
    - `timestamp` (`float`)
    """
    base = Path(filename).stem
    parts = base.split('_')
    if len(parts) < 3:
        raise ValueError(f"Filename '{filename}' does not contain enough underscores.")
    surgery_name = parts[0]
    timestamp_str = parts[-1]
    try:
        timestamp = float(timestamp_str)
    except ValueError:
        raise ValueError(f"Expected a timestamp at the end of '{filename}', but got '{timestamp_str}'.")
    video_name = '_'.join(parts[1:-1])
    return surgery_name, video_name, timestamp

In [16]:
dataroot_path = '../temset'  # Your actual data root path

# Generate microclips
create_microclips(
    dataroot_path=dataroot_path,
    offset=60,    # Number of seconds until the anchor frame
    fps=1,         # Frames per second for the extracted microclip
    n_workers=4    # Number of worker processes to use
)

Invalid duration for ../temset/videos/TEMS-013/142147_VID001.mp4: start_time=0, timestamp=0.0
Invalid duration for ../temset/videos/TEMS-017/162147_VID002.mp4: start_time=0, timestamp=0.0
Invalid duration for ../temset/videos/TEMS-012/100813_VID003.mp4: start_time=0, timestamp=0.0
Invalid duration for ../temset/videos/TEMS-001/152123_VID007.mp4: start_time=0, timestamp=0.0
Invalid duration for ../temset/videos/TEMS-012/100813_VID005.mp4: start_time=0, timestamp=0.0
Invalid duration for ../temset/videos/TEMS-001/152123_VID002.mp4: start_time=0, timestamp=0.0
Invalid duration for ../temset/videos/TEMS-006/115418_VID003.mp4: start_time=0, timestamp=0.0
Invalid duration for ../temset/videos/TEMS-013/142147_VID003.mp4: start_time=0, timestamp=0.0
Invalid duration for ../temset/videos/TEMS-001/152123_VID006.mp4: start_time=0, timestamp=0.0
Invalid duration for ../temset/videos/TEMS-001/152123_VID011.mp4: start_time=0, timestamp=0.0
Invalid duration for ../temset/videos/TEMS-001/152123_VID003

In [17]:
#| hide
import nbdev; nbdev.nbdev_export()